Copyright (c) MONAI Consortium  
Licensed under the Apache License, Version 2.0 (the "License");  
you may not use this file except in compliance with the License.  
You may obtain a copy of the License at  
&nbsp;&nbsp;&nbsp;&nbsp;http://www.apache.org/licenses/LICENSE-2.0  
Unless required by applicable law or agreed to in writing, software  
distributed under the License is distributed on an "AS IS" BASIS,  
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
See the License for the specific language governing permissions and  
limitations under the License. 

# Customize Data Analysis in Auto3DSeg

In this notebook, we will provide a brief example of how to to customize your data analysis pipeline by writing new operations on new metadata.

## Setup environment

In [4]:
!python -c "import monai" || pip install -q "monai-weekly[nibabel]"

## Setup imports

In [5]:
import os
import torch
import tempfile
import nibabel as nib
import numpy as np

from copy import deepcopy
from tqdm import tqdm

from monai.auto3dseg.analyzer import Analyzer
from monai.auto3dseg import (
    SampleOperations,
    SegSummarizer,
    concat_val_to_np,
    datafold_read,
)
from monai.config import print_config
from monai.data import DataLoader, Dataset, create_test_image_3d
from monai.data.utils import no_collation
from monai.transforms import (
    Compose,
    EnsureChannelFirstd,
    EnsureTyped,
    Lambdad,
    LoadImaged,
    Orientationd,
    SqueezeDimd,
    ToDeviced,
)

from monai.utils.enums import DataStatsKeys


def _argmax_if_multichannel(x):
    return torch.argmax(x, dim=0, keepdim=True) if x.shape[0] > 1 else x

from reload_recursive import reload_recursive
import mri_data
import monai_training


print_config()

/home/srs-9/.virtualenvs/monai/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


MONAI version: 1.4.0
Numpy version: 1.26.4
Pytorch version: 2.5.1+cu124
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 46a5272196a6c2590ca2589029eed8e4d56ff008
MONAI __file__: /home/<username>/.virtualenvs/monai/lib/python3.12/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
ITK version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 5.3.2
scikit-image version: NOT INSTALLED or UNKNOWN VERSION.
scipy version: 1.14.1
Pillow version: 11.0.0
Tensorboard version: NOT INSTALLED or UNKNOWN VERSION.
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: NOT INSTALLED or UNKNOWN VERSION.
tqdm version: 4.66.6
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 6.1.0
pandas version: 2.2.3
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: 2.17.2
pynrrd version: NOT INSTALLED or UNKNOWN VERSI

In [6]:
reload_recursive(monai_training)
reload_recursive(mri_data)
import platform
from pathlib import Path
from mri_data.file_manager import scan_3Tpioneer_bids, filter_first_ses
from monai_training.preprocess import DataSetProcesser
from monai_training import training, preprocess

import json

In [7]:
from loguru import logger

logger.add('check.log', level='DEBUG')

2

## Simulate a dataset and Auto3D datalist using MONAI functions

In [8]:
hostname = platform.node()
if hostname == "rhinocampus" or hostname == "ryzen9":
    drive_root = Path("/media/smbshare")
else:
    drive_root = Path("/mnt/h")

projects_root = Path("/home/srs-9/Projects")
msmri_home = projects_root / "ms_mri"
training_work_dirs = msmri_home / "training_work_dirs"

dataroot = drive_root / "3Tpioneer_bids"
work_dir_name = "choroid_pineal_pituitary2"
work_dir = training_work_dirs / work_dir_name
modalities = ["flair", "t1"]
labels = ["choroid_t1_flair", "pineal", "pituitary"]
load_data = 0

In [9]:
if not load_data:
    dataset_proc = DataSetProcesser.new_dataset(dataroot, scan_3Tpioneer_bids, filters=[filter_first_ses])
    dataset_proc.dataset.sort()
    dataset_proc.prepare_labels(labels, suffix_list=["CH", "SRS", "ED", "DT"], resave=True)
    dataset_proc.prepare_images(modalities)

    dataset = dataset_proc.dataset

2024-11-08 16:52:18.755 | INFO     | monai_training.preprocess:prepare_labels:167 - Prepare Labels
2024-11-08 16:52:18.756 | DEBUG    | monai_training.preprocess:prepare_labels:178 - choroid_t1_flair.pineal.pituitary.nii.gz
  0%|          | 0/575 [00:00<?, ?it/s]2024-11-08 16:52:18.759 | DEBUG    | monai_training.preprocess:prepare_labels:196 - Need to create choroid_t1_flair.pineal.pituitary.nii.gz
2024-11-08 16:52:18.759 | INFO     | mri_data.utils:combine_labels:84 - Combining labels for Scan(subid=1001, sesid=20170215)
2024-11-08 16:52:18.765 | DEBUG    | mri_data.file_manager:find_label:401 - Testing choroid_t1_flair-ch.nii.gz
2024-11-08 16:52:18.766 | DEBUG    | mri_data.file_manager:find_label:401 - Testing choroid_t1_flair-srs.nii.gz
2024-11-08 16:52:18.766 | DEBUG    | mri_data.file_manager:find_label:401 - Testing choroid_t1_flair-ed.nii.gz
2024-11-08 16:52:18.767 | DEBUG    | mri_data.file_manager:find_label:401 - Testing choroid_t1_flair-dt.nii.gz
2024-11-08 16:52:18.767 | 

In [10]:
if not load_data:
    dataset = training.assign_conditions(dataset, 0.2)
    dataset[0]

In [11]:
if not load_data:
    training_data = []
    test_data = []

    for scan in dataset:
        if scan.cond == 'tr':
            training_data.append({"image": scan.image, "label": scan.label})
        elif scan.cond == 'ts':
            test_data.append(scan.image)

In [12]:
#? I don't know why they put labels for the test data. the brats segmentation code didn't.
# train_data = [{'image': path + '/flair.nii.gz', 'label': path + '/flair_chp_mask_qced.nii.gz'} for path in train_exams]
# test_data = [{'image': path + '/flair.nii.gz', 'label': path + '/flair_chp_mask_qced.nii.gz'} for path in test_exams]
if not load_data:
    train_data = []
    test_data = []
    for scan in dataset:
        if scan.cond == 'tr' and scan.has_label:
            train_data.append({"image": str(scan.image_path), "label": str(scan.label_path)})
        elif scan.cond == 'ts' and scan.has_label():
            test_data.append({"image": str(scan.image_path), "label": str(scan.label_path)})


    print(f"Train num total: {len(train_data)}")
    print(f"Test num: {len(test_data)}")

Train num total: 24
Test num: 5


In [17]:
if not load_data:
    n_folds = 5
    datalist = {
        "testing": test_data,
        "training": [{"fold": i % n_folds, "image": c["image"], "label": c["label"]} for i,c in enumerate(train_data)]
    }

    if not os.path.isdir(work_dir):
        os.makedirs(work_dir)

    # dataroot_dir = "/mnt/h"
    # if not os.path.isdir(dataroot_dir):
    #     os.makedirs(dataroot_dir)

    datalist_file = os.path.join(work_dir, "datalist.json")
    with open(datalist_file, "w") as f:
        json.dump(datalist, f, indent=4)

In [ ]:
if load_data:
    datalist_file = os.path.join(work_dir, "datalist.json")
    with open(datalist_file, 'r') as f:
        datalist = json.load(f)

    dataset = preprocess.parse_datalist(datalist_file, dataroot)
    dataset.sort()

## Perform analysis on a different image meta data

In [14]:
class DimsAnalyzer(Analyzer):
    def __init__(self, image_key="image", stats_name="user_stats"):
        self.image_key = image_key
        report_format = {"ndims": None}
        super().__init__(stats_name, report_format)

    def __call__(self, data):
        d = dict(data)
        report = deepcopy(self.get_report_format())
        report["ndims"] = d[self.image_key].ndim
        d[self.stats_name] = report
        return d


class DimsSummaryAnalyzer(Analyzer):
    def __init__(self, stats_name="user_stats"):
        report_format = {"ndims": None}
        super().__init__(stats_name, report_format)
        self.update_ops("ndims", SampleOperations())

    def __call__(self, data):
        report = deepcopy(self.get_report_format())
        v_np = concat_val_to_np(data, [self.stats_name, "ndims"])
        report["ndims"] = self.ops["ndims"].evaluate(v_np)
        return report


# it has the three default analyzers (ImageStats, FgImageStats, LabelStats)
summarizer = SegSummarizer("image", "label")
summarizer.add_analyzer(DimsAnalyzer(), DimsSummaryAnalyzer())

In [16]:
def my_analyzer(datalist, dataroot, my_summarizer):
    keys = ["image", "label"]
    transform_list = [
        LoadImaged(keys=keys),
        EnsureChannelFirstd(keys=keys),  # this creates label to be (1,H,W,D)
        Orientationd(keys=keys, axcodes="RAS"),
        EnsureTyped(keys=keys, data_type="tensor"),
        Lambdad(keys="label", func=_argmax_if_multichannel),
        SqueezeDimd(keys=["label"], dim=0),
        ToDeviced(keys=keys, device="cuda" if torch.cuda.is_available() else "cpu"),
        my_summarizer,
    ]

    transform = Compose(transforms=list(filter(None, transform_list)))

    files, _ = datafold_read(datalist=datalist, basedir=dataroot, fold=-1)
    dataset = Dataset(data=files, transform=transform)
    dataloader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=0, collate_fn=no_collation)
    result = {DataStatsKeys.SUMMARY: {}, DataStatsKeys.BY_CASE: []}

    for batch_data in tqdm(dataloader):
        d = batch_data[0]
        stats_by_cases = {
            DataStatsKeys.BY_CASE_IMAGE_PATH: d[DataStatsKeys.BY_CASE_IMAGE_PATH],
            DataStatsKeys.BY_CASE_LABEL_PATH: d[DataStatsKeys.BY_CASE_LABEL_PATH],
            DataStatsKeys.IMAGE_STATS: d[DataStatsKeys.IMAGE_STATS],
            DataStatsKeys.FG_IMAGE_STATS: d[DataStatsKeys.FG_IMAGE_STATS],
            DataStatsKeys.LABEL_STATS: d[DataStatsKeys.LABEL_STATS],
            DataStatsKeys.IMAGE_HISTOGRAM: d[DataStatsKeys.IMAGE_HISTOGRAM],
            "user_stats": d["user_stats"],
        }

        result[DataStatsKeys.BY_CASE].append(stats_by_cases)
    result[DataStatsKeys.SUMMARY] = my_summarizer.summarize(result[DataStatsKeys.BY_CASE])
    return result


result = my_analyzer(datalist, dataroot, summarizer)

  0%|          | 0/24 [00:10<?, ?it/s]


RuntimeError: applying transform <monai.auto3dseg.seg_summarizer.SegSummarizer object at 0x7a14d61078f0>

In [58]:
print(result[DataStatsKeys.BY_CASE][0]["user_stats"])

{'ndims': 4}


In [63]:
work_dir / "datastats_custom.yml"
import yaml
with open(work_dir / "datastats_custom.yml", 'w') as f:
    yaml.dump(result, f, default_flow_style=False)

In [66]:
import json
with open(work_dir / "datastats_custom.json", 'w') as f:
    json.dump(result, f)

In [5]:
with open(work_dir / "datastats_custom.json", 'r') as f:
    result = json.load(f)

In [6]:
result['stats_summary']

{'image_stats': {'shape': {'max': [240, 312, 312],
   'mean': [226.82608695652175, 300.5217391304348, 300.5217391304348],
   'median': [240.0, 300.0, 300.0],
   'min': [189, 300, 300],
   'stdev': [17.03360406070266, 2.447173439907879, 2.447173439907879],
   'percentile': [[189, 300, 300],
    [198, 300, 300],
    [240, 300, 300],
    [240, 312, 312]],
   'percentile_00_5': [189, 300, 300],
   'percentile_10_0': [198, 300, 300],
   'percentile_90_0': [240, 300, 300],
   'percentile_99_5': [240, 312, 312]},
  'channels': {'max': 2,
   'mean': 2.0,
   'median': 2.0,
   'min': 2,
   'stdev': 0.0,
   'percentile': [2, 2, 2, 2],
   'percentile_00_5': 2,
   'percentile_10_0': 2,
   'percentile_90_0': 2,
   'percentile_99_5': 2},
  'cropped_shape': {'max': [240, 301, 307],
   'mean': [226.82608695652175, 297.1304347826087, 296.3478260869565],
   'median': [240.0, 299.0, 300.0],
   'min': [189, 288, 276],
   'stdev': [17.03360406070266, 3.937604080900033, 7.135999949138321],
   'percentile': [

In [24]:
stats_by_case = result['stats_by_cases']
stats_case = stats_by_case[0]
print(stats_case['image_filepath'])
stats_case['label_stats']['label'][0]['image_intensity'][0]['mean']

/mnt/h/3Tpioneer_bids/sub-ms1010/ses-20180208/flair.t1.nii.gz


0.3076651692390442

In [26]:
stats_keys = ['mean','median','max','min']
all_stats = []
for i in range(4):
    stats = {k: list() for k in stats_keys}
    for stats_case in stats_by_case:
        for key in stats_keys:
            stats[key].append(stats_case['label_stats']['label'][i]['image_intensity'][0][key])
    all_stats.append(stats)

In [28]:
import statistics

average_stats = []
for i in range(4):
    avg_stats = {}
    for key in stats_keys:
        avg_stats[key] = statistics.mean(all_stats[i][key])
    average_stats.append(avg_stats)

In [29]:
average_stats

[{'mean': 0.3088977032381555,
  'median': 0.027634781623340172,
  'max': 1.6602173525354136,
  'min': 0.0},
 {'mean': 0.4762034727179486,
  'median': 0.4604956404022548,
  'max': 0.9777869374855704,
  'min': 0.17339564778882524},
 {'mean': 0.4789285815280417,
  'median': 0.46823042501573975,
  'max': 0.7789912845777429,
  'min': 0.2796826006277748},
 {'mean': 0.7886891261391018,
  'median': 0.792717368706413,
  'max': 1.3558782235435818,
  'min': 0.24779999515284662}]

## Add a new stat operation

In [8]:
op = SampleOperations()
# add a new operation
op.update({"sum": np.sum})


class NewDimsSummaryAnalyzer(Analyzer):
    def __init__(self, stats_name="user_stats"):
        report_format = {"ndims": None}
        super().__init__(stats_name, report_format)
        self.update_ops("ndims", op)

    def __call__(self, data):
        report = deepcopy(self.get_report_format())
        v_np = concat_val_to_np(data, [self.stats_name, "ndims"])
        report["ndims"] = self.ops["ndims"].evaluate(v_np)
        return report


summarizer = SegSummarizer("image", "label")
summarizer.add_analyzer(DimsAnalyzer(), NewDimsSummaryAnalyzer())
result = my_analyzer(sim_datalist, sim_dataroot, summarizer)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 30.27it/s]


In [9]:
print(result[DataStatsKeys.SUMMARY]["user_stats"])

{'ndims': {'max': 4, 'mean': 4.0, 'median': 4.0, 'min': 4, 'stdev': 0.0, 'percentile': [4, 4, 4, 4], 'sum': 4, 'percentile_00_5': 4, 'percentile_10_0': 4, 'percentile_90_0': 4, 'percentile_99_5': 4}}
